In [92]:
import os
import sys
import numpy as np
import time
import shutil
import pickle as pkl
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from utils import Config
import gensim
from utils import Config, safe_pickle_dump
import urllib
import feedparser
import urllib.request as libreq
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

model= Doc2Vec.load("d2v.model")
num_of_papers = len(model.docvecs) 
docvecs = []
for i in range(num_of_papers):
    docvecs.append(model.docvecs[i])
docvecs = np.array(docvecs)

In [93]:
category = {} 
db = pkl.load(open(Config.db_path, 'rb'))
for k,v in db.items():
    category[k] = v['tags'][0]['term']
set_of_category =  set(category.values())
colors = []
root = int(np.ceil(np.power(len(set_of_category),1/4)))
for a in range(1,1+root):
    for b in range(1,1+root):
        for c in range(1,1+root):
            for d in range(1,1+root):
                colors.append((1/root*a,1/root*b,1/root*c,1/root*d))

corr = dict(zip(list(set_of_category),colors[:len(set_of_category)]))

In [94]:
def TSE_visualization(X,query_dict,output_file=None):
    plt.figure(figsize=(16, 16)) 
    X_embedded = TSNE(n_components=2, n_iter=1000, perplexity=40, verbose=2, random_state=23).fit_transform(X)
    color = np.array([corr[category[query]] for k,query in query_dict.items()])
    label = np.array([legend[category[query]] for k, query in query_dict.items()])
    plt.scatter(X_embedded[:,0],X_embedded[:,1],color=color)
    plt.legend()
    if output_file:
        plt.savefig(output_file)
    plt.show()

In [ ]:
# visualization of doc2vec
doc2vec_dict = {i: model.docvecs.index_to_doctag(i)[:-2] for i in range(num_of_papers)}
TSE_visualization(docvecs,doc2vec_dict,"doc2vec_visual.png")

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 2761 samples in 0.004s...
[t-SNE] Computed neighbors for 2761 samples in 0.821s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2761
[t-SNE] Computed conditional probabilities for sample 2000 / 2761
[t-SNE] Computed conditional probabilities for sample 2761 / 2761
[t-SNE] Mean sigma: 4.346590
[t-SNE] Computed conditional probabilities in 0.247s
[t-SNE] Iteration 50: error = 78.6955719, gradient norm = 0.0181344 (50 iterations in 4.259s)
[t-SNE] Iteration 100: error = 75.7591782, gradient norm = 0.0132952 (50 iterations in 3.291s)
[t-SNE] Iteration 150: error = 75.6563950, gradient norm = 0.0140651 (50 iterations in 2.937s)
[t-SNE] Iteration 200: error = 75.6512299, gradient norm = 0.0202712 (50 iterations in 3.041s)
[t-SNE] Iteration 250: error = 75.6296921, gradient norm = 0.0151828 (50 iterations in 3.143s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 75.629692
[t-SNE] Iteration 300: err

In [ ]:
# visualization of BERT
db = pkl.load(open('bert_out_big.p', 'rb'))
X = np.array(list(db.values()))
X = X / np.linalg.norm(X, axis=0)
pids = list(db.keys())
ds = -np.asarray(np.dot(X, X.T))
bert_dict = dict(zip(range(num_of_papers), pids))
TSE_visualization(ds, bert_dict,"bert_visual.png")